In [65]:
import pandas as pd
import numpy as np
np.random.seed(123)  # for reproducibility
 
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, LeakyReLU, PReLU, LocallyConnected2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras import backend as K
import tensorflow as tf

from scipy.ndimage import rotate

from random import uniform
from numpy.random import choice

from matplotlib import pyplot as plt
%matplotlib inline

from randomized_relu import randomized_relu
from drop_activation import drop_activation
from keras.utils.generic_utils import get_custom_objects

get_custom_objects().update({'randomized_relu': Activation(randomized_relu)})
get_custom_objects().update({'drop_activation': Activation(drop_activation)})

In [66]:
# define mirror functions
def mirror_X(input_X):
    return input_X.reshape(-1,96,96)[:, :, list(range(95, -1, -1))].reshape(-1, 96*96)
    
def mirror_y(input_y, n_point):
    output_y = input_y.copy()
    # y coordinate stays the same, x coordinate is 96-x
    output_y[:,list(range(0,n_point-1,2))] = 95 - input_y[:,list(range(0,n_point-1,2))]
    return output_y

In [67]:
# mirror function to transform X
def rotate_X(input_X, degree):
    
    # create an array of the same size
    output_X = np.zeros((input_X.shape), dtype=np.float64)
    
    # for each image, use sklearn's rotate function to rotate image
    # this somehow changes pixel values, cap value between 0 and 255
    for i in range(output_X.shape[0]):
        output_X[i] = np.clip(rotate(input_X.reshape(-1,96,96)[i], degree, reshape=False, mode='constant', \
                                       cval=150).reshape(96*96), 0, 255)  
    return output_X
        
    
# mirror function to transform y
def rotate_y(input_y, degree, n_point):
    
    # create an array of same size
    output_y = np.zeros((input_y.shape), dtype=np.float64)
    
    # define rotation angle and center of rotation
    theta = np.radians(degree)
    center_x = 95/2
    center_y = 95/2

    for i in range(output_y.shape[0]):
        
        # obtain x and y coordinates from dataset
        x = input_y[i][np.arange(0,n_point-1,2)]
        y = input_y[i][np.arange(1,n_point,2)]
        
        # use rotation matrix to rotate coordinates around center
        x2 = np.cos(theta) * (x - center_x) + np.sin(theta) * (y - center_y) + center_x
        y2 = -np.sin(theta) * (x - center_x) + np.cos(theta) * (y - center_y) + center_y
        
        # put x and y back in original shape
        output_y[i] = np.vstack((x2,y2)).transpose().flatten()
        
    return output_y

In [68]:
# define function - only need to transform X
def reduce_contrast(input_X, weight):
    return (weight * input_X) + (1 - weight) * input_X.mean()

In [69]:
# prep data
def prep_data(X, y, points, val_pct):
    
    # model with specific points 
    y_model = y[:, points]

    # get index of non-missing labels
    index = ~np.isnan(y_model).any(axis=1)

    # filter out missing labels on training and validation data
    X_model = X[index]
    y_model = y_model[index]
    
    # create training and validation set (80/20 split)
    X_train, X_val = X_model[:int(X_model.shape[0]*(1-val_pct))], X_model[int(X_model.shape[0]*(1-val_pct)):]
    y_train, y_val = y_model[:int(X_model.shape[0]*(1-val_pct))], y_model[int(X_model.shape[0]*(1-val_pct)):]

    # training data transformation
    degree1 = -10
    degree2 = 10
    n_point = len(points)
    percent = 0.9

    X_rotated = np.zeros((X_train.shape), dtype=np.float64)
    X_rotated[range(0, len(X_train), 2)] = rotate_X(X_train[range(0, len(X_train), 2)], degree1)
    X_rotated[range(1, len(X_train), 2)] = rotate_X(X_train[range(1, len(X_train), 2)], degree2)

    y_rotated = np.zeros((y_train.shape), dtype=np.float64)
    y_rotated[range(0, len(y_train), 2)] = rotate_y(y_train[range(0, len(y_train), 2)], degree1, n_point)
    y_rotated[range(1, len(y_train), 2)] = rotate_y(y_train[range(1, len(y_train), 2)], degree2, n_point)   

    X_rotated = np.concatenate((X_train, X_rotated), axis=0)
    y_rotated = np.concatenate((y_train, y_rotated), axis=0)
    
    X_contrast = X_rotated.copy()
    X_contrast[range(1, len(X_rotated), 2)] = reduce_contrast(X_rotated[range(1, len(X_rotated), 2)], percent)
    
    return X_contrast.reshape(-1,96,96,1), y_rotated, X_val.reshape(-1,96,96,1), y_val

# Model 1 with Locally Connected Layer

In [6]:
# define RMSE
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# create model - use random search
def create_model(x_train, y_train, n_point):

    n_conv_layer = 6
    conv_filter1 = 8
    conv_filter2 = 32
    conv_filter3 = 64
    conv_filter4 = 64
    conv_filter5 = 64
    conv_filter6 = 128
    
    n_dense_layer = 3
    dense_neuron1 = 128
    dense_neuron2 = 96
    dense_neuron3 = 32
     
    description = 'ConvLayer-{}-Filter-{}-{}-{}-{}-{}-{}-LC-Y-DenseLayer-{}-Neuron-{}-{}-{}-Activation-PReLU'.format( \
            n_conv_layer, conv_filter1, conv_filter2, conv_filter3, conv_filter4, conv_filter5, conv_filter6, \
            n_dense_layer, dense_neuron1, dense_neuron2, dense_neuron3)
    
    
    # define model
    model = Sequential()
    
    # conv layer 1
    model.add(Conv2D(conv_filter1, (3, 3), strides=(1,1), padding='same', \
                     input_shape=(96,96,1), data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    # conv layer 2
    model.add(Conv2D(conv_filter2, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # conv layer 3
    model.add(Conv2D(conv_filter3, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
        
    # lc layer 4
    model.add(LocallyConnected2D(conv_filter4, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
            
    # lc layer 5
    model.add(LocallyConnected2D(conv_filter5, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # lc layer 6
    model.add(LocallyConnected2D(conv_filter6, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    
    model.add(Flatten())
    
    # fully connected layer 1
    model.add(Dense(dense_neuron1))
    model.add(PReLU())
    
    # fully connected layer 2
    model.add(Dense(dense_neuron2))
    model.add(PReLU())
        
    # fully connected layer 3
    model.add(Dense(dense_neuron3))
    model.add(PReLU())   
    
    # output layer
    model.add(Dense(n_point))


    # compile model
    model.compile(loss='mean_squared_error', metrics=[rmse], optimizer='adam')

    return model, description

In [70]:
# load data
X = np.load('X.npy')
y = np.load('y.npy')

In [71]:
# create data
X_model1, y_model1, X_model1_val, y_model1_val = prep_data(X, y, points = [0, 1, 2, 3, 20, 21, 28, 29], val_pct=0)
X_model2, y_model2, X_model2_val, y_model2_val = prep_data(X, y, points=[4,5,6,7,8,9,10,11,12,13,14,15,16, \
                                                                         17,18,19,22,23,24,25,26,27], val_pct=0)

In [10]:
model, description = create_model(X_model1, y_model1, 8)
print(description)

# fit model
result = model.fit(X_model1, y_model1, batch_size=64, epochs=30, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


ConvLayer-6-Filter-8-32-64-96-128-256-LC-Y-DenseLayer-3-Neuron-256-128-96-Activation-PReLU
Train on 13300 samples, validate on 700 samples
Epoch 1/30
 - 60s - loss: 76.2056 - rmse: 5.9939 - val_loss: 25.2953 - val_rmse: 5.0122
Epoch 2/30
 - 47s - loss: 16.4057 - rmse: 4.0078 - val_loss: 33.8393 - val_rmse: 5.8081
Epoch 3/30
 - 47s - loss: 10.6988 - rmse: 3.2246 - val_loss: 15.2679 - val_rmse: 3.8821
Epoch 4/30
 - 47s - loss: 8.0427 - rmse: 2.7925 - val_loss: 14.2383 - val_rmse: 3.7484
Epoch 5/30
 - 47s - loss: 6.7731 - rmse: 2.5582 - val_loss: 13.9652 - val_rmse: 3.7141
Epoch 6/30
 - 47s - loss: 5.8511 - rmse: 2.3742 - val_loss: 7.7733 - val_rmse: 2.7370
Epoch 7/30
 - 47s - loss: 5.2839 - rmse: 2.2598 - val_loss: 7.1785 - val_rmse: 2.6200
Epoch 8/30
 - 47s - loss: 4.8136 - rmse: 2.1523 - val_loss: 7.7471 - val_rmse: 2.7382
Epoch 9/30
 - 47s - loss: 4.5278 - rmse: 2.0709 - val_loss: 8.6832 - val_rmse: 2.9113
Epoch 10/30
 - 47s - loss: 3.8447 - rmse: 1.9123 - val_loss: 7.2353 - val_rmse:

In [ ]:
# fit model
result = model.fit(X_model1, y_model1, batch_size=128, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)

Train on 13300 samples, validate on 700 samples
Epoch 1/20


In [ ]:
del result
del model
K.clear_session()

In [ ]:
model, description = create_model(X_model1, y_model1, 8, 'PReLU')
print(description)

# fit model
result = model.fit(X_model2, y_model2, batch_size=64, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


# Model 2

In [10]:
# define RMSE
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# create model - use random search
def create_model(x_train, y_train, n_point):

    n_conv_layer = 6
    conv_filter1 = 8
    conv_filter2 = 64
    conv_filter3 = 96
    conv_filter4 = 128
    conv_filter5 = 256
    conv_filter6 = 324
    
    n_dense_layer = 3
    dense_neuron1 = 256
    dense_neuron2 = 96
    dense_neuron3 = 64
     
    description = 'ConvLayer-{}-Filter-{}-{}-{}-{}-{}-{}-LC-N-DenseLayer-{}-Neuron-{}-{}-{}-Activation-PReLU'.format( \
            n_conv_layer, conv_filter1, conv_filter2, conv_filter3, conv_filter4, conv_filter5, conv_filter6, \
            n_dense_layer, dense_neuron1, dense_neuron2, dense_neuron3)
    
    
    # define model
    model = Sequential()
    
    # conv layer 1
    model.add(Conv2D(conv_filter1, (3, 3), strides=(1,1), padding='same', \
                     input_shape=(96,96,1), data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    # conv layer 2
    model.add(Conv2D(conv_filter2, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # conv layer 3
    model.add(Conv2D(conv_filter3, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
        
    # conv layer 4
    model.add(Conv2D(conv_filter4, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
            
    # conv layer 5
    model.add(Conv2D(conv_filter5, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # conv layer 6
    model.add(Conv2D(conv_filter6, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    
    model.add(Flatten())
    
    # fully connected layer 1
    model.add(Dense(dense_neuron1))
    model.add(PReLU())
    
    # fully connected layer 2
    model.add(Dense(dense_neuron2))
    model.add(PReLU())
        
    # fully connected layer 3
    model.add(Dense(dense_neuron3))
    model.add(PReLU())   
    
    # output layer
    model.add(Dense(n_point))


    # compile model
    model.compile(loss='mean_squared_error', metrics=[rmse], optimizer='adam')

    return model, description

In [9]:
model, description = create_model(X_model1, y_model1, 8)
print(description)

# fit model
result = model.fit(X_model1, y_model1, batch_size=64, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


ConvLayer-6-Filter-8-64-96-128-256-324-LC-N-DenseLayer-3-Neuron-256-96-64-Activation-PReLU
Train on 13300 samples, validate on 700 samples
Epoch 1/20
 - 16s - loss: 127.2125 - rmse: 7.0941 - val_loss: 61.3578 - val_rmse: 7.8253
Epoch 2/20
 - 14s - loss: 11.4598 - rmse: 3.3332 - val_loss: 21.1109 - val_rmse: 4.5802
Epoch 3/20
 - 14s - loss: 8.2392 - rmse: 2.8231 - val_loss: 12.8376 - val_rmse: 3.5523
Epoch 4/20
 - 14s - loss: 6.3533 - rmse: 2.4735 - val_loss: 14.7145 - val_rmse: 3.8157
Epoch 5/20
 - 14s - loss: 5.3996 - rmse: 2.2771 - val_loss: 13.0159 - val_rmse: 3.5854
Epoch 6/20
 - 14s - loss: 4.7822 - rmse: 2.1422 - val_loss: 17.4576 - val_rmse: 4.1563
Epoch 7/20
 - 14s - loss: 4.4293 - rmse: 2.0604 - val_loss: 6.0184 - val_rmse: 2.3872
Epoch 8/20
 - 14s - loss: 3.8953 - rmse: 1.9342 - val_loss: 5.7027 - val_rmse: 2.3209
Epoch 9/20
 - 14s - loss: 3.7031 - rmse: 1.8704 - val_loss: 5.7806 - val_rmse: 2.3353
Epoch 10/20
 - 14s - loss: 3.3118 - rmse: 1.7768 - val_loss: 14.3425 - val_rms

In [10]:
# fit model
result = model.fit(X_model1, y_model1, batch_size=128, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


Train on 13300 samples, validate on 700 samples
Epoch 1/20
 - 13s - loss: 1.6183 - rmse: 1.2232 - val_loss: 3.5878 - val_rmse: 1.8450
Epoch 2/20
 - 13s - loss: 1.2700 - rmse: 1.0758 - val_loss: 8.5349 - val_rmse: 2.9042
Epoch 3/20
 - 13s - loss: 1.0864 - rmse: 0.9897 - val_loss: 15.5250 - val_rmse: 3.9326
Epoch 4/20
 - 13s - loss: 1.0957 - rmse: 0.9913 - val_loss: 3.6496 - val_rmse: 1.8627
Epoch 5/20
 - 13s - loss: 1.0557 - rmse: 0.9647 - val_loss: 6.8592 - val_rmse: 2.6035
Epoch 6/20
 - 13s - loss: 1.0499 - rmse: 0.9702 - val_loss: 25.6027 - val_rmse: 5.0565
Epoch 7/20
 - 13s - loss: 1.0891 - rmse: 0.9927 - val_loss: 4.0972 - val_rmse: 1.9876
Epoch 8/20
 - 13s - loss: 1.0065 - rmse: 0.9381 - val_loss: 14.2987 - val_rmse: 3.7746
Epoch 9/20
 - 13s - loss: 1.0739 - rmse: 0.9761 - val_loss: 43.7463 - val_rmse: 6.6122
Epoch 10/20
 - 13s - loss: 0.9549 - rmse: 0.9095 - val_loss: 3.1366 - val_rmse: 1.7185
Epoch 11/20
 - 13s - loss: 0.9369 - rmse: 0.8994 - val_loss: 38.2023 - val_rmse: 6.1791

In [12]:
# fit model
result = model.fit(X_model1, y_model1, batch_size=512, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)

Train on 13300 samples, validate on 700 samples
Epoch 1/20
 - 12s - loss: 0.4537 - rmse: 0.6271 - val_loss: 3.8809 - val_rmse: 1.9480
Epoch 2/20
 - 12s - loss: 0.4101 - rmse: 0.5921 - val_loss: 3.3716 - val_rmse: 1.8056
Epoch 3/20
 - 12s - loss: 0.4320 - rmse: 0.6016 - val_loss: 2.9837 - val_rmse: 1.6902
Epoch 4/20
 - 12s - loss: 0.5640 - rmse: 0.7193 - val_loss: 3.1445 - val_rmse: 1.7450
Epoch 5/20
 - 12s - loss: 0.4810 - rmse: 0.6480 - val_loss: 3.1847 - val_rmse: 1.7500
Epoch 6/20
 - 12s - loss: 0.4025 - rmse: 0.5830 - val_loss: 2.9234 - val_rmse: 1.6763
Epoch 7/20
 - 12s - loss: 0.4373 - rmse: 0.6189 - val_loss: 5.8012 - val_rmse: 2.3979
Epoch 8/20
 - 12s - loss: 0.5165 - rmse: 0.6630 - val_loss: 3.4368 - val_rmse: 1.8289
Epoch 9/20
 - 12s - loss: 0.4375 - rmse: 0.6131 - val_loss: 2.9037 - val_rmse: 1.6672
Epoch 10/20
 - 12s - loss: 0.4122 - rmse: 0.5862 - val_loss: 2.7866 - val_rmse: 1.6316
Epoch 11/20
 - 12s - loss: 0.4244 - rmse: 0.6032 - val_loss: 3.0385 - val_rmse: 1.7067
Epoc

In [13]:
# save model
model.save('model1_4pts_try2.h5')

del result
del model
K.clear_session()

In [16]:
model, description = create_model(X_model1, y_model1, 22)
print(description)

# fit model
result = model.fit(X_model2, y_model2, batch_size=64, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


ConvLayer-6-Filter-8-64-96-128-256-324-LC-N-DenseLayer-3-Neuron-256-96-64-Activation-PReLU
Train on 4094 samples, validate on 216 samples
Epoch 1/20
 - 6s - loss: 268.7913 - rmse: 11.4141 - val_loss: 43.6134 - val_rmse: 6.6018
Epoch 2/20
 - 4s - loss: 15.0408 - rmse: 3.8657 - val_loss: 24.4405 - val_rmse: 4.9417
Epoch 3/20
 - 4s - loss: 11.5847 - rmse: 3.3919 - val_loss: 21.2558 - val_rmse: 4.6062
Epoch 4/20
 - 4s - loss: 8.5489 - rmse: 2.9143 - val_loss: 22.2491 - val_rmse: 4.7144
Epoch 5/20
 - 4s - loss: 6.9431 - rmse: 2.6258 - val_loss: 16.5177 - val_rmse: 4.0610
Epoch 6/20
 - 4s - loss: 6.1373 - rmse: 2.4663 - val_loss: 11.2440 - val_rmse: 3.3495
Epoch 7/20
 - 4s - loss: 5.4151 - rmse: 2.3199 - val_loss: 9.7001 - val_rmse: 3.1107
Epoch 8/20
 - 4s - loss: 5.1197 - rmse: 2.2541 - val_loss: 6.8471 - val_rmse: 2.6133
Epoch 9/20
 - 4s - loss: 4.9189 - rmse: 2.2119 - val_loss: 5.3716 - val_rmse: 2.3140
Epoch 10/20
 - 4s - loss: 4.1296 - rmse: 2.0276 - val_loss: 4.9754 - val_rmse: 2.2264


In [17]:
# fit model
result = model.fit(X_model2, y_model2, batch_size=128, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


Train on 4094 samples, validate on 216 samples
Epoch 1/20
 - 4s - loss: 1.9526 - rmse: 1.3902 - val_loss: 3.1571 - val_rmse: 1.7767
Epoch 2/20
 - 4s - loss: 1.9819 - rmse: 1.4027 - val_loss: 3.9222 - val_rmse: 1.9804
Epoch 3/20
 - 4s - loss: 1.7261 - rmse: 1.3103 - val_loss: 4.8529 - val_rmse: 2.2029
Epoch 4/20
 - 4s - loss: 1.6497 - rmse: 1.2806 - val_loss: 2.9476 - val_rmse: 1.7166
Epoch 5/20
 - 4s - loss: 1.5294 - rmse: 1.2348 - val_loss: 3.0575 - val_rmse: 1.7484
Epoch 6/20
 - 4s - loss: 1.6861 - rmse: 1.2947 - val_loss: 3.1700 - val_rmse: 1.7800
Epoch 7/20
 - 4s - loss: 1.7148 - rmse: 1.3037 - val_loss: 2.8924 - val_rmse: 1.7007
Epoch 8/20
 - 4s - loss: 1.6029 - rmse: 1.2602 - val_loss: 3.3898 - val_rmse: 1.8411
Epoch 9/20
 - 4s - loss: 1.4674 - rmse: 1.2081 - val_loss: 2.3717 - val_rmse: 1.5400
Epoch 10/20
 - 4s - loss: 1.3057 - rmse: 1.1411 - val_loss: 2.3758 - val_rmse: 1.5413
Epoch 11/20
 - 4s - loss: 1.3394 - rmse: 1.1554 - val_loss: 2.4680 - val_rmse: 1.5709
Epoch 12/20
 - 4

In [18]:
# fit model
result = model.fit(X_model2, y_model2, batch_size=512, epochs=40, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


Train on 4094 samples, validate on 216 samples
Epoch 1/40
 - 5s - loss: 1.1966 - rmse: 1.0919 - val_loss: 2.0231 - val_rmse: 1.4224
Epoch 2/40
 - 4s - loss: 1.0177 - rmse: 1.0087 - val_loss: 1.8646 - val_rmse: 1.3655
Epoch 3/40
 - 4s - loss: 0.9202 - rmse: 0.9588 - val_loss: 2.2468 - val_rmse: 1.4989
Epoch 4/40
 - 4s - loss: 0.9011 - rmse: 0.9484 - val_loss: 2.5214 - val_rmse: 1.5879
Epoch 5/40
 - 4s - loss: 0.9109 - rmse: 0.9528 - val_loss: 2.8759 - val_rmse: 1.6959
Epoch 6/40
 - 4s - loss: 0.8362 - rmse: 0.9138 - val_loss: 3.0566 - val_rmse: 1.7483
Epoch 7/40
 - 4s - loss: 0.8338 - rmse: 0.9127 - val_loss: 3.1126 - val_rmse: 1.7643
Epoch 8/40
 - 4s - loss: 0.7859 - rmse: 0.8864 - val_loss: 3.0566 - val_rmse: 1.7483
Epoch 9/40
 - 4s - loss: 0.8221 - rmse: 0.9058 - val_loss: 3.0675 - val_rmse: 1.7514
Epoch 10/40
 - 4s - loss: 0.7798 - rmse: 0.8824 - val_loss: 3.5213 - val_rmse: 1.8765
Epoch 11/40
 - 4s - loss: 0.8362 - rmse: 0.9141 - val_loss: 3.3070 - val_rmse: 1.8185
Epoch 12/40
 - 4

In [19]:
# save model
model.save('model2_11pts_try2.h5')

del result
del model
K.clear_session()

# Model 3

In [72]:
# define RMSE
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# create model - use random search
def create_model(n_point):

    n_conv_layer = 6
    conv_filter1 = 28
    conv_filter2 = 64
    conv_filter3 = 64
    conv_filter4 = 64
    conv_filter5 = 128
    conv_filter6 = 128
    
    n_dense_layer = 3
    dense_neuron1 = 256
    dense_neuron2 = 96
    dense_neuron3 = 64
     
    description = 'ConvLayer-{}-Filter-{}-{}-{}-{}-{}-{}-LC-N-DenseLayer-{}-Neuron-{}-{}-{}-Activation-PReLU'.format( \
            n_conv_layer, conv_filter1, conv_filter2, conv_filter3, conv_filter4, conv_filter5, conv_filter6, \
            n_dense_layer, dense_neuron1, dense_neuron2, dense_neuron3)
    
    
    # define model
    model = Sequential()
    
    # conv layer 1
    model.add(Conv2D(conv_filter1, (3, 3), strides=(1,1), padding='same', \
                     input_shape=(96,96,1), data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    # conv layer 2
    model.add(Conv2D(conv_filter2, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # conv layer 3
    model.add(Conv2D(conv_filter3, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
        
    # conv layer 4
    model.add(Conv2D(conv_filter4, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
            
    # conv layer 5
    model.add(Conv2D(conv_filter5, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # conv layer 6
    model.add(Conv2D(conv_filter6, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    
    model.add(Flatten())
    
    # fully connected layer 1
    model.add(Dense(dense_neuron1))
    model.add(PReLU())
    
    # fully connected layer 2
    model.add(Dense(dense_neuron2))
    model.add(PReLU())
        
    # fully connected layer 3
    model.add(Dense(dense_neuron3))
    model.add(PReLU())   
    
    # output layer
    model.add(Dense(n_point))


    # compile model
    model.compile(loss='mean_squared_error', metrics=[rmse], optimizer='adam')

    return model, description

In [73]:
model, description = create_model(8)
print(description)

# fit model
result = model.fit(X_model1, y_model1, batch_size=64, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


ConvLayer-6-Filter-28-64-64-64-128-128-LC-N-DenseLayer-3-Neuron-256-96-64-Activation-PReLU
Train on 13300 samples, validate on 700 samples
Epoch 1/20
 - 16s - loss: 178.8321 - rmse: 8.3293 - val_loss: 36.1222 - val_rmse: 6.0020
Epoch 2/20
 - 14s - loss: 13.2840 - rmse: 3.5989 - val_loss: 22.0750 - val_rmse: 4.6842
Epoch 3/20
 - 14s - loss: 9.1611 - rmse: 2.9778 - val_loss: 20.8995 - val_rmse: 4.5554
Epoch 4/20
 - 14s - loss: 7.1707 - rmse: 2.6330 - val_loss: 14.8749 - val_rmse: 3.8386
Epoch 5/20
 - 14s - loss: 6.0294 - rmse: 2.4082 - val_loss: 18.4252 - val_rmse: 4.2784
Epoch 6/20
 - 14s - loss: 5.3828 - rmse: 2.2702 - val_loss: 9.3457 - val_rmse: 3.0191
Epoch 7/20
 - 14s - loss: 4.7527 - rmse: 2.1371 - val_loss: 6.1298 - val_rmse: 2.4073
Epoch 8/20
 - 14s - loss: 4.1697 - rmse: 2.0039 - val_loss: 6.6994 - val_rmse: 2.5338
Epoch 9/20
 - 14s - loss: 4.0364 - rmse: 1.9612 - val_loss: 6.8710 - val_rmse: 2.5559
Epoch 10/20
 - 14s - loss: 3.4850 - rmse: 1.8247 - val_loss: 13.6678 - val_rmse

In [74]:
# fit model
result = model.fit(X_model1, y_model1, batch_size=128, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


Train on 13300 samples, validate on 700 samples
Epoch 1/20
 - 14s - loss: 1.7605 - rmse: 1.2915 - val_loss: 3.5416 - val_rmse: 1.8277
Epoch 2/20
 - 13s - loss: 1.4729 - rmse: 1.1731 - val_loss: 3.8034 - val_rmse: 1.9019
Epoch 3/20
 - 13s - loss: 1.2889 - rmse: 1.0957 - val_loss: 11.6742 - val_rmse: 3.4052
Epoch 4/20
 - 13s - loss: 1.3320 - rmse: 1.1098 - val_loss: 5.4884 - val_rmse: 2.3138
Epoch 5/20
 - 13s - loss: 1.4302 - rmse: 1.1419 - val_loss: 7.7478 - val_rmse: 2.7670
Epoch 6/20
 - 13s - loss: 1.3750 - rmse: 1.1327 - val_loss: 3.9291 - val_rmse: 1.9418
Epoch 7/20
 - 13s - loss: 1.3555 - rmse: 1.1192 - val_loss: 3.9104 - val_rmse: 1.9313
Epoch 8/20
 - 13s - loss: 1.2457 - rmse: 1.0648 - val_loss: 17.9951 - val_rmse: 4.2370
Epoch 9/20
 - 13s - loss: 1.3646 - rmse: 1.1203 - val_loss: 11.4973 - val_rmse: 3.3790
Epoch 10/20
 - 13s - loss: 1.3104 - rmse: 1.0954 - val_loss: 3.7333 - val_rmse: 1.8871
Epoch 11/20
 - 13s - loss: 1.1545 - rmse: 1.0207 - val_loss: 37.2141 - val_rmse: 6.0990


In [75]:
# fit model
result = model.fit(X_model1, y_model1, batch_size=512, epochs=40, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)

Train on 13300 samples, validate on 700 samples
Epoch 1/40
 - 14s - loss: 0.7968 - rmse: 0.8687 - val_loss: 3.2706 - val_rmse: 1.7700
Epoch 2/40
 - 12s - loss: 0.6348 - rmse: 0.7675 - val_loss: 3.2183 - val_rmse: 1.7551
Epoch 3/40
 - 12s - loss: 0.6454 - rmse: 0.7684 - val_loss: 3.3428 - val_rmse: 1.7932
Epoch 4/40
 - 12s - loss: 0.6149 - rmse: 0.7461 - val_loss: 3.2385 - val_rmse: 1.7583
Epoch 5/40
 - 12s - loss: 0.6429 - rmse: 0.7733 - val_loss: 3.2967 - val_rmse: 1.7768
Epoch 6/40
 - 12s - loss: 0.5758 - rmse: 0.7263 - val_loss: 3.5561 - val_rmse: 1.8526
Epoch 7/40
 - 12s - loss: 0.5662 - rmse: 0.7076 - val_loss: 3.4257 - val_rmse: 1.8156
Epoch 8/40
 - 12s - loss: 0.5770 - rmse: 0.7296 - val_loss: 4.1471 - val_rmse: 2.0074
Epoch 9/40
 - 12s - loss: 0.5561 - rmse: 0.7116 - val_loss: 3.0775 - val_rmse: 1.7136
Epoch 10/40
 - 12s - loss: 0.5174 - rmse: 0.6794 - val_loss: 3.2231 - val_rmse: 1.7573
Epoch 11/40
 - 12s - loss: 0.5868 - rmse: 0.7355 - val_loss: 3.1886 - val_rmse: 1.7469
Epoc

In [76]:
# save model
model.save('model1_4pts_try3.h5')

del result
del model
K.clear_session()

In [77]:
model, description = create_model(22)
print(description)

# fit model
result = model.fit(X_model2, y_model2, batch_size=64, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


ConvLayer-6-Filter-28-64-64-64-128-128-LC-N-DenseLayer-3-Neuron-256-96-64-Activation-PReLU
Train on 4094 samples, validate on 216 samples
Epoch 1/20
 - 6s - loss: 346.8132 - rmse: 13.2479 - val_loss: 59.2346 - val_rmse: 7.6914
Epoch 2/20
 - 4s - loss: 13.3733 - rmse: 3.6459 - val_loss: 16.1351 - val_rmse: 4.0123
Epoch 3/20
 - 4s - loss: 10.3172 - rmse: 3.1924 - val_loss: 12.8059 - val_rmse: 3.5746
Epoch 4/20
 - 4s - loss: 7.6619 - rmse: 2.7574 - val_loss: 8.3556 - val_rmse: 2.8795
Epoch 5/20
 - 4s - loss: 6.6297 - rmse: 2.5671 - val_loss: 6.4464 - val_rmse: 2.5288
Epoch 6/20
 - 4s - loss: 5.6887 - rmse: 2.3806 - val_loss: 6.9661 - val_rmse: 2.6295
Epoch 7/20
 - 4s - loss: 4.9917 - rmse: 2.2270 - val_loss: 7.2271 - val_rmse: 2.6817
Epoch 8/20
 - 4s - loss: 4.4230 - rmse: 2.0980 - val_loss: 5.6428 - val_rmse: 2.3696
Epoch 9/20
 - 4s - loss: 4.2698 - rmse: 2.0597 - val_loss: 5.2305 - val_rmse: 2.2831
Epoch 10/20
 - 4s - loss: 4.2622 - rmse: 2.0556 - val_loss: 4.5285 - val_rmse: 2.1257
Epo

In [78]:
# fit model
result = model.fit(X_model2, y_model2, batch_size=128, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


Train on 4094 samples, validate on 216 samples
Epoch 1/20
 - 4s - loss: 1.8261 - rmse: 1.3487 - val_loss: 2.9409 - val_rmse: 1.7147
Epoch 2/20
 - 4s - loss: 1.7602 - rmse: 1.3248 - val_loss: 4.0355 - val_rmse: 2.0088
Epoch 3/20
 - 4s - loss: 1.6999 - rmse: 1.3006 - val_loss: 3.6356 - val_rmse: 1.9065
Epoch 4/20
 - 4s - loss: 1.5802 - rmse: 1.2541 - val_loss: 2.7702 - val_rmse: 1.6642
Epoch 5/20
 - 4s - loss: 1.5731 - rmse: 1.2515 - val_loss: 2.6142 - val_rmse: 1.6166
Epoch 6/20
 - 4s - loss: 1.6020 - rmse: 1.2624 - val_loss: 2.6184 - val_rmse: 1.6178
Epoch 7/20
 - 4s - loss: 1.3996 - rmse: 1.1817 - val_loss: 2.5499 - val_rmse: 1.5966
Epoch 8/20
 - 4s - loss: 1.4296 - rmse: 1.1943 - val_loss: 3.0127 - val_rmse: 1.7355
Epoch 9/20
 - 4s - loss: 1.3587 - rmse: 1.1645 - val_loss: 2.9858 - val_rmse: 1.7277
Epoch 10/20
 - 4s - loss: 1.3218 - rmse: 1.1483 - val_loss: 2.4274 - val_rmse: 1.5578
Epoch 11/20
 - 4s - loss: 1.3802 - rmse: 1.1722 - val_loss: 2.5641 - val_rmse: 1.6009
Epoch 12/20
 - 4

In [79]:
# fit model
result = model.fit(X_model2, y_model2, batch_size=512, epochs=40, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


Train on 4094 samples, validate on 216 samples
Epoch 1/40
 - 5s - loss: 1.4013 - rmse: 1.1793 - val_loss: 2.4199 - val_rmse: 1.5556
Epoch 2/40
 - 4s - loss: 1.2083 - rmse: 1.0966 - val_loss: 2.2607 - val_rmse: 1.5036
Epoch 3/40
 - 4s - loss: 1.0533 - rmse: 1.0259 - val_loss: 2.0961 - val_rmse: 1.4478
Epoch 4/40
 - 4s - loss: 0.9841 - rmse: 0.9916 - val_loss: 2.1005 - val_rmse: 1.4493
Epoch 5/40
 - 4s - loss: 0.9405 - rmse: 0.9697 - val_loss: 2.0947 - val_rmse: 1.4473
Epoch 6/40
 - 4s - loss: 0.9298 - rmse: 0.9638 - val_loss: 2.2340 - val_rmse: 1.4946
Epoch 7/40
 - 4s - loss: 0.9206 - rmse: 0.9593 - val_loss: 2.2395 - val_rmse: 1.4965
Epoch 8/40
 - 4s - loss: 0.8936 - rmse: 0.9452 - val_loss: 2.4103 - val_rmse: 1.5525
Epoch 9/40
 - 4s - loss: 0.9263 - rmse: 0.9620 - val_loss: 2.6860 - val_rmse: 1.6389
Epoch 10/40
 - 4s - loss: 0.9003 - rmse: 0.9481 - val_loss: 2.6379 - val_rmse: 1.6241
Epoch 11/40
 - 4s - loss: 0.8827 - rmse: 0.9394 - val_loss: 2.6381 - val_rmse: 1.6242
Epoch 12/40
 - 4

In [80]:
# save model
model.save('model2_11pts_try3.h5')

del result
del model
K.clear_session()

# Score Test Data and Ensemble

In [82]:
import pandas as pd
import numpy as np

In [83]:
# read in data
training = pd.read_csv("all/training.csv")
test = pd.read_csv('all/test.csv')
lookup = pd.read_csv('all/IdLookupTable.csv')
sample_submission = pd.read_csv('all/SampleSubmission.csv')
print(test.shape)
print(lookup.shape)
print(sample_submission.shape)

(1783, 2)
(27124, 4)
(27124, 2)


In [84]:
# split out image values
image = []
for i in range(test.shape[0]):
    img = test['Image'][i].split(' ')
    img = ['0' if j == '' else j for j in img]
    image.append(img)
    
X_test = np.array(image, dtype = 'float').reshape(-1,96,96,1)
X_test.shape

(1783, 96, 96, 1)

In [11]:
# Model 2
# load models
model1 = load_model('model1_4pts_try2.h5', custom_objects={'rmse': rmse})
model2 = load_model('model2_11pts_try2.h5', custom_objects={'rmse': rmse})

# make prediction on test set
y_test_model1 = model1.predict(X_test, batch_size=None, verbose=0, steps=None)
y_test_model2 = model2.predict(X_test, batch_size=None, verbose=0, steps=None)

# combine outputs from 2 models
pts1 = [0,1,2,3,20,21,28,29]
pts2 = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,22,23,24,25,26,27]
y_test = np.zeros((1783,30), dtype=np.float64)
y_test[:, pts1] = y_test_model1
y_test[:, pts2] = y_test_model2

# prepare submission
submission0 = pd.DataFrame(y_test, columns=training.columns[:-1])
submission1 = pd.concat([test['ImageId'], submission0], axis=1)
submission2 = pd.melt(submission1, id_vars=['ImageId'], value_vars=training.columns[:-1]).rename \
    (columns={"variable": "FeatureName", "value": "Location"})
submission3 = pd.merge(lookup.drop(columns=['Location']), submission2, on=['ImageId','FeatureName'], how='left')
submission4 = submission3[['RowId', 'Location']]
submission4['Location'][submission4['Location']>96]=96
submission4.to_csv('submission_2.csv', index=False)
print(submission4.shape)
submission4.head()

In [85]:
# Model 3
# load models
model1 = load_model('model1_4pts_try3.h5', custom_objects={'rmse': rmse})
model2 = load_model('model2_11pts_try3.h5', custom_objects={'rmse': rmse})

# make prediction on test set
y_test_model1 = model1.predict(X_test, batch_size=None, verbose=0, steps=None)
y_test_model2 = model2.predict(X_test, batch_size=None, verbose=0, steps=None)

# combine outputs from 2 models
pts1 = [0,1,2,3,20,21,28,29]
pts2 = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,22,23,24,25,26,27]
y_test = np.zeros((1783,30), dtype=np.float64)
y_test[:, pts1] = y_test_model1
y_test[:, pts2] = y_test_model2

# prepare submission
submission0 = pd.DataFrame(y_test, columns=training.columns[:-1])
submission1 = pd.concat([test['ImageId'], submission0], axis=1)
submission2 = pd.melt(submission1, id_vars=['ImageId'], value_vars=training.columns[:-1]).rename \
    (columns={"variable": "FeatureName", "value": "Location"})
submission3 = pd.merge(lookup.drop(columns=['Location']), submission2, on=['ImageId','FeatureName'], how='left')
submission4 = submission3[['RowId', 'Location']]
submission4['Location'][submission4['Location']>96]=96
submission4.to_csv('submission_3.csv', index=False)
print(submission4.shape)
submission4.head()

C:\Users\Filabuster\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(27124, 2)


,RowId,Location
0,1,67.125801
1,2,38.217121
2,3,29.540987
3,4,37.156696
4,5,60.071815


In [86]:
submission1 = pd.read_csv("submission.csv")
submission2 = pd.read_csv("submission_2.csv")
submission3 = pd.read_csv("submission_3.csv")

In [87]:
ensemble0 = pd.merge(submission1, submission2, on=['RowId'], how='inner')
ensemble = pd.merge(ensemble0, submission3, on=['RowId'], how='inner')
ensemble.head()

,RowId,Location_x,Location_y,Location
0,1,67.415268,68.196732,67.125801
1,2,38.005962,38.641598,38.217121
2,3,29.572035,30.426266,29.540987
3,4,36.070862,36.990479,37.156696
4,5,60.440517,58.371372,60.071815


In [90]:
ensemble['Location'] = ensemble[['Location_x', 'Location_y', 'Location']].mean(axis=1)
ensemble.drop(columns=['Location_x', 'Location_y'], inplace=True)

In [91]:
ensemble.head()

,RowId,Location
0,1,67.730422
1,2,38.311929
2,3,29.948243
3,4,36.600229
4,5,59.479930


In [92]:
ensemble.to_csv('submission_ensemble2.csv', index=False)